In [35]:
import re
from datetime import datetime

# Define file paths
raw_file_path = "work_data.txt"
cleaned_file_path = "cleaned_chat_data.txt"

# Read all lines from the file
with open(raw_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Define the regex pattern for a timestamp at the start of a line
timestamp_pattern = r"^\d{2}\.\d{2}\.\d{4}, \d{2}:\d{2}"

# 1. Find the latest date in the entire file and filter lines
latest_date = None
lines_with_dates = []

for line in lines:
    if re.match(timestamp_pattern, line):
        date_str = line.split(',')[0].strip()
        line_date = datetime.strptime(date_str, "%d.%m.%Y").date()
        
        # If we find a new, later date, reset our list of lines
        if latest_date is None or line_date > latest_date:
            latest_date = line_date
            lines_with_dates = [line]
        # If it's the same latest date, add the line
        elif line_date == latest_date:
            lines_with_dates.append(line)
    else:
        # If the line doesn't have a timestamp, it's a continuation.
        # Add it to the most recent date's lines.
        if lines_with_dates:
            lines_with_dates.append(line)

if not lines_with_dates:
    print("No valid lines with a date/time format were found in the file.")
    exit()

# 2. Process the filtered lines to build the complete messages
processed_messages = []
current_message = ""

for line in lines_with_dates:
    # Check if the line starts with a timestamp
    if re.match(timestamp_pattern, line):
        # Save the previous message and start a new one
        if current_message:
            processed_messages.append(current_message.strip())
        current_message = line.strip()
    else:
        # If it does not have a timestamp, it's a continuation
        current_message +=  ' ' + line.strip()

# Add the final message to the list after the loop finishes
if current_message:
    processed_messages.append(current_message.strip())



In [36]:
# Save the final, processed messages to the new file
with open(cleaned_file_path, "w", encoding='utf-8') as file:
    for message in processed_messages:
        file.write(message + "\n")

print(f"Successfully processed {len(processed_messages)} messages for the date: {latest_date.strftime('%d.%m.%Y')}")
print(f"Cleaned data has been saved to '{cleaned_file_path}'.")

Successfully processed 9 messages for the date: 17.09.2025
Cleaned data has been saved to 'cleaned_chat_data.txt'.


In [37]:
import pandas as pd

In [38]:
# Path to the cleaned text file from Step 1
cleaned_file_path = "cleaned_chat_data.txt"

# A dictionary to store the parsed data, keyed by (name, date)
data_dict = {}

# Regex patterns for extraction
# This pattern extracts the date, time, and the rest of the message content
message_pattern = re.compile(r"(\d{2}\.\d{2}\.\d{4}),\s+(\d{2}:\d{2})\s+-\s+(.*)")

# Define the correct list of vehicles and their regex patterns for case-insensitive matching
vehicles = [
    r'Iveco\s?prelată', r'Iveco', r'Prelata', r'Prelată', r'Peugeot\s?Expert',
    r'Peugeot', r'Macara', r'Personală', r'Personala', r'TK', r'Renault\s?TK'
]
vehicle_pattern = re.compile(r"|".join(vehicles), re.IGNORECASE)

# Keywords for start and end times
start_keywords = re.compile(r'Am\.?\s?început|Am\.?\s?inceput', re.IGNORECASE)
end_keywords = re.compile(r'Am\.?\s?terminat', re.IGNORECASE)

# Read the cleaned text file line by line
with open(cleaned_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue
        
        # Match the entire line against the message pattern
        match = message_pattern.match(line)
        if not match:
            continue
            
        date_str, time_str, message_content = match.groups()
        
        # Split message_content to find the name and the rest of the message
        # The name is the part before the first colon
        name_end_index = message_content.find(":")
        if name_end_index != -1:
            name_str = message_content[:name_end_index].strip()
            # The message starts after the colon and the following space
            message_body = message_content[name_end_index+1:].strip()
        else:
            name_str = "Unknown"
            message_body = message_content.strip()

        # Extract Ora intrare and Ora iesire
        ora_intrare = None
        if start_keywords.search(message_body):
            ora_intrare = time_str
            
        ora_iesire = None
        if end_keywords.search(message_body):
            ora_iesire = time_str
            
        # Extract Masina and Locatii
        masina = None
        # Remove keywords before looking for Masina, to prevent false positives
        body_without_keywords = start_keywords.sub('', message_body)
        body_without_keywords = end_keywords.sub('', body_without_keywords)

        masina_found = False
        # Find all vehicles in the message body
        for m in vehicle_pattern.finditer(body_without_keywords):
            found_vehicle = m.group(0).strip()
            # Handle special case for 'Personala' or 'Personală'
            if 'personal' in found_vehicle.lower():
                try:
                    words = message_body.split()
                    personal_word_index = words.index(found_vehicle)
                    # Check for word after "Personala"
                    if personal_word_index + 1 < len(words):
                        masina = f"{found_vehicle} {words[personal_word_index + 1]}"
                    else:
                        masina = found_vehicle
                    masina_found = True
                    break
                except (ValueError, IndexError):
                    pass
            
            # For other vehicles, just take the first one found
            if masina is None:
                masina = found_vehicle
                masina_found = True
                break

        # Extract Locatii by removing masina and keywords
        locatii = message_body
        locatii = start_keywords.sub('', locatii)
        locatii = end_keywords.sub('', locatii)
        
        # Remove the Masina text from the Locatii string
        if masina_found and masina is not None:
            locatii = locatii.replace(masina, '').strip()
        
        # Clean up remaining text in Locatii
        locatii = re.sub(r'^\s*[-,\s:]+', '', locatii).strip()
        locatii = re.sub(r'[-,\s:]+$', '', locatii).strip()
        if not locatii:
            locatii = None

        # Create a unique key for each person on each day
        key = (name_str, date_str)
        
        # Use the key to update or create a new entry in the dictionary
        if key not in data_dict:
            data_dict[key] = {
                'Numele': name_str,
                'Data': date_str,
                'Ora intrare': ora_intrare,
                'Ora iesire': ora_iesire,
                'Masina': masina,
                'Locatii': locatii
            }
        else:
            if ora_intrare:
                data_dict[key]['Ora intrare'] = ora_intrare
            if ora_iesire:
                data_dict[key]['Ora iesire'] = ora_iesire
            if masina:
                data_dict[key]['Masina'] = masina
            if locatii:
                data_dict[key]['Locatii'] = locatii

# Convert the dictionary to a DataFrame
pontaj_df = pd.DataFrame(list(data_dict.values()))

# Convert string dates to datetime objects for calculation
pontaj_df['Data'] = pd.to_datetime(pontaj_df['Data'], format='%d.%m.%Y')
pontaj_df['Entry_Time_DT'] = pd.to_datetime(pontaj_df['Ora intrare'], format='%H:%M', errors='coerce')
pontaj_df['Exit_Time_DT'] = pd.to_datetime(pontaj_df['Ora iesire'], format='%H:%M', errors='coerce')

# Calculate Timp total de lucru
pontaj_df['Timp total de lucru'] = (pontaj_df['Exit_Time_DT'] - pontaj_df['Entry_Time_DT']).dt.total_seconds() / 3600
pontaj_df['Timp total de lucru'] = pontaj_df['Timp total de lucru'].round(2)

# Drop intermediate columns
final_columns = ['Numele', 'Data', 'Ora intrare', 'Ora iesire', 'Masina', 'Locatii', 'Timp total de lucru']
pontaj_df = pontaj_df[final_columns]

# Display the final DataFrame to review the result
#print("Final DataFrame:")
# print(pontaj_df.to_string())

In [39]:
pontaj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Numele               8 non-null      object        
 1   Data                 8 non-null      datetime64[ns]
 2   Ora intrare          8 non-null      object        
 3   Ora iesire           0 non-null      object        
 4   Masina               3 non-null      object        
 5   Locatii              7 non-null      object        
 6   Timp total de lucru  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 580.0+ bytes


In [40]:
import os

# # --- This code block assumes you have already created the 'final_df' from Step 2 ---

# # Get the month and year from the 'Data' column of the DataFrame
# # We can use the first entry since all entries are from the same day
# if not pontaj_df.empty:
#     latest_date_from_df = pontaj_df['Data'].iloc[0]
    
#     # Format the date to "Month Year" for the filename
#     excel_file_name = latest_date_from_df.strftime('%B %Y.xlsx')
    
#     # Check if the file already exists and provide a message
#     if os.path.exists(excel_file_name):
#         print(f"File '{excel_file_name}' already exists. It will be overwritten.")

#     # Save the DataFrame to an Excel file
#     pontaj_df.to_excel(excel_file_name, index=False)
    
#     print(f"DataFrame successfully saved to '{excel_file_name}'")

# else:
#     print("The DataFrame is empty. No file was saved.")

In [41]:
def update_or_create_excel(pontaj_df):
    """
    Update existing Excel file with new data or create a new one if it doesn't exist
    
    Args:
        pontaj_df: DataFrame containing the new data to add/update
    """
    
    if pontaj_df.empty:
        print("DataFrame is empty. No file operations performed.")
        return
    
    # Get the month and year from the 'Data' column of the DataFrame
    # We can use the first entry since all entries are from the same day
    latest_date_from_df = pontaj_df['Data'].iloc[0]
    
    # Format the date to "Month Year" for the filename
    excel_file_name = latest_date_from_df.strftime('%B %Y.xlsx')
    
    try:
        if os.path.exists(excel_file_name):
            print(f"File '{excel_file_name}' already exists. Updating with new data...")
            
            # Read the existing Excel file
            existing_df = pd.read_excel(excel_file_name)
            
            # Check if 'Data' column exists in both DataFrames for proper merging
            if 'Data' in existing_df.columns and 'Data' in pontaj_df.columns:
                
                # Convert 'Data' columns to datetime for proper comparison
                existing_df['Data'] = pd.to_datetime(existing_df['Data'])
                pontaj_df['Data'] = pd.to_datetime(pontaj_df['Data'])
                
                # Get dates that are in the new data
                new_dates = pontaj_df['Data'].unique()
                
                # Remove rows from existing data that have the same dates as new data
                # This prevents duplicates and ensures we have the latest data
                existing_df_filtered = existing_df[~existing_df['Data'].isin(new_dates)]
                
                # Combine existing data (without duplicated dates) with new data
                updated_df = pd.concat([existing_df_filtered, pontaj_df], ignore_index=True)
                
                # Sort by date to maintain chronological order
                updated_df = updated_df.sort_values('Data').reset_index(drop=True)
                
                print(f"Updated data: {len(existing_df_filtered)} existing records + {len(pontaj_df)} new records = {len(updated_df)} total records")
                
            else:
                # If 'Data' column doesn't exist or structure is different, append new data
                print("Warning: Column structure doesn't match. Appending new data to existing file.")
                updated_df = pd.concat([existing_df, pontaj_df], ignore_index=True)
            
            # Save the updated DataFrame back to Excel
            updated_df.to_excel(excel_file_name, index=False)
            print(f"✓ File '{excel_file_name}' updated successfully!")
            
        else:
            # File doesn't exist, create a new one
            print(f"File '{excel_file_name}' does not exist. Creating new file...")
            pontaj_df.to_excel(excel_file_name, index=False)
            print(f"✓ New file '{excel_file_name}' created successfully!")
            
        # Display summary information
        print(f"\nFile Details:")
        print(f"  Filename: {excel_file_name}")
        print(f"  Location: {os.path.abspath(excel_file_name)}")
        print(f"  Records: {len(updated_df) if os.path.exists(excel_file_name) and 'updated_df' in locals() else len(pontaj_df)}")
        
    except Exception as e:
        print(f"Error processing Excel file: {e}")
        
        # Fallback: save with timestamp to avoid data loss
        fallback_filename = f"pontaj_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        pontaj_df.to_excel(fallback_filename, index=False)
        print(f"Data saved to fallback file: {fallback_filename}")

In [43]:
if not pontaj_df.empty:
    update_or_create_excel(pontaj_df)

File 'September 2025.xlsx' already exists. Updating with new data...
Updated data: 10 existing records + 8 new records = 18 total records
✓ File 'September 2025.xlsx' updated successfully!

File Details:
  Filename: September 2025.xlsx
  Location: e:\Deeplearning.ai DA\Modulul 3 - python\Pontaj\September 2025.xlsx
  Records: 18
